In [ ]:
!pip install transformers tokenizers sentencepiece protobuf huggingface_hub accelerate

In [1]:
!git clone https://github.com/huggingface/diffusers

Cloning into 'diffusers'...
remote: Enumerating objects: 71474, done.
remote: Counting objects: 100% (9899/9899), done.
remote: Compressing objects: 100% (761/761), done.
remote: Total 71474 (delta 9599), reused 9182 (delta 9121), pack-reused 61575 (from 1)
Receiving objects: 100% (71474/71474), 49.76 MiB | 6.58 MiB/s, done.
Resolving deltas: 100% (53028/53028), done.
Updating files: 100% (1581/1581), done.


In [2]:
!pip install ./diffusers

Processing ./diffusers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.31.0.dev0-py3-none-any.whl size=2799834 sha256=fa1601c4a14ca1444f128e7a586b407a027d7c1933b3f5078e36fc5a2459e3c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-o6mjh3y3/wheels/47/b1/28/98902d05eddc64208d6a467cad03b9f75d3644537ecb576eb4
Successfully built diffusers

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
## Tests
1. Run the default config and verify that its working as intended
2. Run cfg and see its updating the batch size to 2
3. Run cfg++ and see its updating the batch size to 2 and scale is b/w 0 and 1
4. Run APG and sweep the hyperparams.
5. Run APG-sample and instead use it on predicted sample rather than noise
6. Run SEG, implement the SEG attn processor.

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
import torch
from pipeline_flux import FluxPipeline

pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16)
pipe.to('cuda')

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

FluxPipeline {
  "_class_name": "FluxPipeline",
  "_diffusers_version": "0.31.0.dev0",
  "_name_or_path": "black-forest-labs/FLUX.1-dev",
  "scheduler": [
    "diffusers",
    "FlowMatchEulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "T5EncoderModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "T5TokenizerFast"
  ],
  "transformer": [
    "diffusers",
    "FluxTransformer2DModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [3]:
prompt = "A cat holding a sign that says hello world"
image = pipe(
    prompt,
    height=1024,
    width=1024,
    guidance_scale=3.5,
    num_inference_steps=50,
    max_sequence_length=512,
    generator=torch.Generator("cuda").manual_seed(0),
    debug=True,
    compile=False,
    guidance_mode='cfg',
    guidance_weight=7.5
).images[0]

  0%|          | 0/50 [00:00<?, ?it/s]

latents shape torch.Size([2, 4096, 64]), txt_ids, torch.Size([512, 3]), img_ids torch.Size([4096, 3]), guidance torch.Size([1]) timesteps torch.Size([50]), prompt_embeds torch.Size([1, 512, 4096]), pooled torch.Size([1, 768])


RuntimeError: The size of tensor a (8704) must match the size of tensor b (4608) at non-singleton dimension 2

In [ ]:
batch size 2
latents shape torch.Size([2, 4096, 64]),
txt_ids, torch.Size([512, 3]),
img_ids torch.Size([4096, 3]),
guidance torch.Size([2])
timesteps torch.Size([50]),
prompt_embeds torch.Size([2, 512, 4096]),
pooled torch.Size([2, 768])